In [13]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [14]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [15]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [16]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [18]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [25]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [26]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [27]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [28]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("Mother Teresa Mission Higher Secondary School", "https://www.mtmhss.edu.in/")

Found links: {'relevant_links': ['https://www.mtmhss.edu.in/Kidwai-Nagar/kidwai-nagar.aspx', 'https://www.mtmhss.edu.in/Koyla-Nagar/koyla-nagar.aspx', 'https://www.mtmhss.edu.in/Puffins/Puffins.aspx']}


# Welcome to Mother Teresa Mission Higher Secondary School
*Where learning is a journey and joy is the destination!*

---

### About Us
Nestled in the heart of Kanpur, the **Mother Teresa Mission Higher Secondary School (MTMHSS)** is not just an educational institution; it's a community celebration of knowledge, values, and creativity! 🏫✨

**Vision**: We’re on a mission to mold citizens armed with a sense of justice and a pinch of leadership – because who doesn’t love a little global challenge with their study notes? 🌍💪

**Mission**: We cherish each student's unique talents, quirks, and inspiring ability to nap during class, because every individual brings something special to the table – even if that something is just a really good pencil case! ✏️🎒

---

### Our Philosophical Foundation
In our world, **diversity** is not just a buzzword – it’s our middle name! 🎉 We acknowledge the rich tapestry of backgrounds, interests, and cultural muscles that make us all unique. That means we embrace everyone from budding Einsteins to aspiring pop stars, and we ensure they all get along (mostly over pizza)!

---

### Academic Achievements
- **Affiliation**: We’re proud to be affiliated with the Council for the Indian School Certificate Examination that believes in an engaging curriculum tailored for individual success. 🎓
- **Curriculum**: From ICSE to ISC, we've got programs that'll make your head spin in the best way possible – and don’t worry, we also teach exam techniques! 😂📚
  
---

### Extracurricular Extravaganza 🎉
Because learning shouldn't end at the classroom door:
- 🏅 **Sports & Games**: Because running around the field is *totally* better than sitting still!
- 🎤 **Music & Dance**: Unleash your inner pop star or ballet dancer, we promise no judgment – only applause!
- 🧘‍♂️ **Yoga**: Find your zen and melt away stress from homework with some well-timed “downward dog.”

---

### Join Our Fun-tastic Team!
Looking to jump into a career with a lot of heart? We are always on the lookout for passionate staff who share our love for laugh-filled classrooms! Whether you can educate, motivate, or just keep the laughter flowing, **we’ve got a spot for you**! 🌟👩‍🏫

**Available Positions**: Check our website for job openings, as we never say “no” to talent! 

---

### Upcoming Events
Stay tuned for events where students showcase their talents—because who doesn’t love the smell of popcorn, a stage, and students doing dramatic interpretations of "Hamlet"? 🍿🎭

---

### Get in Touch
Remember, life is just a classroom if you don’t engage!  
For updates, job opportunities, or to just send us love letters:  
- **Email Us**: [mtmhss95@gmail.com](mailto:mtmhss95@gmail.com)
- **Facebook**: [Mother Teresa Mission School FB](#)
- **Instagram**: [@mtmhss_fun](#)
- **LinkedIn**: [MTMHS LinkedIn Page](#)

**Location**: Kidwai Nagar, Kanpur!

Join us where we believe that **learning is a joy and each student is a superstar!** 🌟

---
*Disclaimer: No actual pop stars or cookie-cutter educational models were harmed in the making of this brochure.*